Initialise code for google colab

Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_addons as tfa
print(tfa.__version__)

In [ ]:
!pip install tensorflow==2.11
!pip install tensorflow-addons==0.19

Create data base files under google colab environment

In [ ]:
!unzip -q '/content/drive/MyDrive/data_equalize.zip' -d '/content/'

For Luc because my archive made on mac create a __MACOSX folder

In [ ]:
%rm -rf /content/__MACOSX

Define working directory to our jupyter repertory:
* because path to the different repertories (./data, ./output...) are define relatevly to jupyter one
* let import _mypath which add ./lib to python path in order to import our own define libraries


In [ ]:
%cd /content/drive/MyDrive/covid-19-xRay/jupyter

In [15]:
import _mypath
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications import DenseNet201


In [16]:
from database.path_origin_data import build_data_paths 
from database.path_origin_data import lung_name, infection_name
from database.path_origin_data import train_name, test_name, valid_name
from database.path_origin_data import normal_name, covid_name, no_covid_name
from database.path_origin_data import images_name, lung_mask_name, infection_mask_name

from database.dataset import build_dataset

Build paths and variables for reading data base hierarchy

In [17]:
# for local use
db_path = '../data_equalize'


In [ ]:
# for google colab use
db_path = '/content/data_equalize'


In [18]:
output_path = os.path.join('..', 'output')
if not os.path.exists(output_path):
    os.makedirs(output_path, exist_ok=True)


Structure to manage paths in data base

In [19]:
data_paths = build_data_paths()
idx = pd.IndexSlice


Create tf Dataset

In [20]:
paths = data_paths['path']

ds_train = build_dataset(db_path, paths, db=[lung_name], ds=[train_name])
ds_test = build_dataset(db_path, paths, db=[lung_name], ds=[test_name])
ds_valid = build_dataset(db_path, paths, db=[lung_name], ds=[valid_name])
ds_train


Found 6849 files belonging to 1 classes.
Found 7658 files belonging to 1 classes.
Found 7208 files belonging to 1 classes.
Found 2140 files belonging to 1 classes.
Found 2394 files belonging to 1 classes.
Found 2253 files belonging to 1 classes.
Found 1712 files belonging to 1 classes.
Found 1902 files belonging to 1 classes.
Found 1802 files belonging to 1 classes.


<ConcatenateDataset element_spec=(TensorSpec(shape=(256, 256, 1), dtype=tf.float32, name=None), TensorSpec(shape=(3,), dtype=tf.int32, name=None))>

In [21]:
def repeat_chan(img, label):
  return tf.repeat(img, 3, axis=-1), label

ds_train = ds_train.map(repeat_chan)
ds_test = ds_test.map(repeat_chan)
ds_valid = ds_valid.map(repeat_chan)
ds_train

<MapDataset element_spec=(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(3,), dtype=tf.int32, name=None))>

Model parameters

In [27]:
batch_size = 4
image_size = 256
input_shape = (image_size, image_size, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 100

label_smoothing = 0.1

# data augmentation
flip = "horizontal"
rotation_factor = 0.1
zoom_height_factor = 0.2
zoom_width_factor = 0.2

mlp_head_units = [128, 64]  # Size of the dense layers of the final classifier



Data augmentation

In [28]:
norm_layer = layers.Normalization()

ds_norm = ds_train.map(lambda img, l: img).take(1000).batch(batch_size)
norm_layer.adapt(ds_norm)
print(norm_layer.mean)
print(norm_layer.variance)

data_augmentation_seq = keras.Sequential(
    [
      norm_layer,
      # layers.Resizing(image_size, image_size),
      # layers.RandomFlip(flip),
      # layers.RandomRotation(rotation_factor),
      # layers.RandomZoom(height_factor=zoom_height_factor, width_factor=zoom_width_factor),
    ],
    name='data_augmentation'
)

tf.Tensor([[[[123.78828 123.78828 123.78828]]]], shape=(1, 1, 1, 3), dtype=float32)
tf.Tensor([[[[5787.6235 5787.6235 5787.6235]]]], shape=(1, 1, 1, 3), dtype=float32)


Pretrained model

In [24]:
from myLayers.mlp import mlp

# preproc = tf.keras.applications.vgg16.preprocess_input(augmented)
baseModel = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
# preproc = tf.keras.applications.vgg19.preprocess_input(input)
# baseModel = VGG19(weights="imagenet", include_top=False,input_tensor=layers.Input(shape=(image_size, image_size, 3)))
# preproc = tf.keras.applications.DenseNet201.preprocess_input(input)
# baseModel = DenseNet201(weights="imagenet", include_top=False,input_tensor=layers.Input(shape=(image_size, image_size, 3)))
for layer in baseModel.layers:
    layer.trainable = False


inputs = layers.Input(shape=input_shape)
augmented = data_augmentation_seq(inputs)
base = baseModel(augmented)
# construct the head of the model that will be placed on top of the
head = layers.AveragePooling2D(pool_size=(8, 8))(base)
head = layers.Flatten(name="flatten")(head)
features = mlp(head, hidden_units=mlp_head_units, dropout_rate=0.5)
# Classify outputs.
softmax = layers.Dense(3, activation='softmax', kernel_initializer='random_normal')(features)

# Create the Keras model.
model = keras.Model(inputs=inputs, outputs=softmax)


In [25]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, None, None, 3)    7         
 al)                                                             
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 average_pooling2d_2 (Averag  (None, 1, 1, 512)        0         
 ePooling2D)                                                     
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               6566

In [29]:
from run_exp.standard import run_experiment

checkpoint_filepath = os.path.join(output_path, "checkpoint_transfer")
run_experiment(model,
              ds_train, ds_test, ds_valid,
              batch_size=batch_size, num_epochs=num_epochs,
              learning_rate=learning_rate, weight_decay=weight_decay,
              checkpoint_filepath=checkpoint_filepath,
              from_logits=False, label_smoothing=label_smoothing)

Epoch 1/100


2022-12-21 18:14:14.483397: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 6895 of 21715
2022-12-21 18:14:24.480651: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 14154 of 21715
2022-12-21 18:14:34.481670: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 19560 of 21715
2022-12-21 18:14:37.934767: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


 258/5429 [>.............................] - ETA: 1:47:24 - loss: 1.0340 - accuracy: 0.4748